In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\lec_7\Project.toml`
Precompiling project...
  ✓ OrdinaryDiffEq
  ✓ DelayDiffEq
  ✓ ControlSystems
  3 dependencies successfully precompiled in 515 seconds (141 already precompiled)


In [ ]:
using LinearAlgebra
using Plots

## Define Dynamics

In [ ]:
h = 0.1   # time step
A = [ 1 h; 0 1 ]
B = [ 0.5*h*h; h ]

In [ ]:
n = 2     # number of state
m = 1     # number of controls
Tfinal = 5.0 # final time #try larger values
N = Int(Tfinal/h) + 1
t_hist = Array(range(0, h*(N-1), step=h))

## Initial state and LQR variables

In [2]:
x0 = [ 1.0; 0 ]
Q = 1.0 * I(2)
R = 0.1
Qn = 1.0 * I(2)

# Julia repeat() ref : https://www.geeksforgeeks.org/creating-array-with-repeated-elements-in-julia-repeat-method/
repeat(x0, N)

LoadError: UndefVarError: I not defined